<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/classification_grouping_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import openai
import zipfile
import json
import os
import json

In [2]:
!pip install openai

In [3]:
!pip install --upgrade openai

  Using cached openai-1.81.0-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.81.0-py3-none-any.whl (717 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [4]:
openai.api_key = ""

In [ ]:
merged_ranking_path = "merged_ranking.txt"
ranking_res_zip = "/content/ranking_results.zip"

with zipfile.ZipFile(ranking_res_zip, 'r') as z, open(merged_ranking_path, 'w', encoding='utf-8') as out:
    for info in z.infolist():
        if info.filename.endswith('.txt'):
            out.write(f"===== {info.filename} =====\n")
            with z.open(info) as f:
                for raw in f:
                    out.write(raw.decode('utf-8', errors='replace'))
            out.write("\n\n")

print(f"Done — all .txt merged into `{merged_ranking_path}`")


In [ ]:
# 1. Update these paths
obs_res_zip        = '/content/json_results.zip'
extract_dir     = '/content/extracted_jsons/json_results'
merged_json_path = 'merged_obs_extraction.json'

# 2. Extract JSON files
with zipfile.ZipFile(obs_res_zip, 'r') as zf:
    zf.extractall(extract_dir)

# 3. Merge into one dictionary, extending duplicates’ observables
merged = {}
for filename in os.listdir(extract_dir):
    # only process .json files
    if not filename.lower().endswith('.json'):
        continue

    full_path = os.path.join(extract_dir, filename)
    with open(full_path, 'r') as f:
        data = json.load(f)

    for md_name, doc in data.items():
        if md_name not in merged:
            merged[md_name] = doc
        else:
            # extend observables list from both
            merged[md_name]["observables"].extend(doc.get("observables", []))
            # you can merge other fields here if needed

# 4. Save the merged JSON
with open(merged_json_path, 'w') as f:
    json.dump(merged, f, indent=2)

In [5]:
# Suppose your file was named “data.json”
with open('merged_obs_extraction.json','r') as f:
    merged_obs_extraction = json.load(f)

In [6]:
# Extract, lowercase, and dedupe classification values
unique_classifications = {
    obs['classification'].lower()
    for doc in merged_obs_extraction.values()
    for obs in doc.get('observables', [])
    if 'classification' in obs
}

In [7]:
unique_classifications.update(['api',
 'api call',
 'api function',
 'api hook',
 'archive file',
 'attack effect',
 'attack technique',
 'authentication credential',
 'authentication technique',
 'boot record',
 'c2 server',
 'circuit breaker',
 'code snippet',
 'command',
 'command line argument',
 'command message',
 'command parameters',
 'command sequence',
 'command value',
 'command/utility',
 'communication channel',
 'communication interface',
 'communication port',
 'communication priority level',
 'communication protocol',
 'config/data file',
 'configuration data',
 'configuration file',
 'configuration parameter',
 'credential',
 'credential list',
 'data',
 'data block',
 'data manipulation',
 'data transfer',
 'database server',
 'device',
 'device configuration value',
 'device state',
 'device status parameter',
 'distributed control system',
 'dll file',
 'document',
 'domain name',
 'driver file',
 'dynamic-link library',
 'electrical device',
 'email message',
 'error state',
 'event log',
 'executable command',
 'executable file',
 'exploit payload',
 'exploit technique',
 'facility',
 'file',
 'file type',
 'file/script',
 'firewall rule',
 'firewall utility',
 'firmware',
 'frequency converter drive',
 'function block memory location',
 'hardware component',
 'hex value',
 'hmi device',
 'host device',
 'http request',
 'ics address',
 'ics application',
 'ics command',
 'ics command message',
 'ics communication payload',
 'ics communication protocol',
 'ics configuration',
 'ics configuration file',
 'ics controller',
 'ics data',
 'ics data block recording',
 'ics device',
 'ics device parameter',
 'ics field',
 'ics function block',
 'ics gateway',
 'ics i/o snapshot',
 'ics network communication',
 'ics object',
 'ics parameter',
 'ics payload component',
 'ics process data',
 'ics project file',
 'ics protocol',
 'ics protocol field',
 'ics protocol module',
 'ics protocol parameter',
 'ics protocol value',
 'ics software module',
 'ics systems',
 'ics tag',
 'identifier constant',
 'impact condition',
 'industrial communication protocol',
 'industrial control device',
 'industrial control module',
 'industrial control system',
 'industrial control system (ics) device',
 'industrial control system device',
 'industrial control system platform',
 'industrial controller (cpu)',
 'industrial network bus',
 'integrity check routine',
 'log files',
 'malicious file',
 'malicious ics function block',
 'malicious logic',
 'malicious software',
 'malware',
 'malware module',
 'manufacturing facility',
 'medical product',
 'memory access pattern',
 'memory access request',
 'memory write attempt',
 'module',
 'monitor',
 'network communication',
 'network communication technique',
 'network configuration',
 'network connection',
 'network data',
 'network device',
 'network entity',
 'network identifier',
 'network packet',
 'network port',
 'network protocol',
 'network segment',
 'network service',
 'network service/port',
 'network traffic',
 'network traffic event',
 'opc server',
 'opc ua address space',
 'opc ua server',
 'operating system api',
 'operational mode',
 'os system call',
 'packaging tool',
 'payload file',
 'plc',
 'plc block',
 'plc code',
 'plc code component',
 'plc cpu',
 'plc data block',
 'plc data block content (magic value)',
 'plc device',
 'plc function block',
 'plc i/o image',
 'plc operating system',
 'plc organization block',
 'plc program organization unit',
 'plc runtime constraint',
 'process',
 'process name list',
 'process termination',
 'profibus communication module',
 'program transfer operation',
 'project file',
 'protective relay',
 'protocol',
 'protocol field',
 'protocol object data',
 'protocol version',
 'proxy server',
 'python function call',
 'registry key',
 'remote access service',
 'remote terminal unit',
 'removable media',
 'resource identifier',
 'safety instrumented system',
 'script',
 'script engine/interpreter',
 'script file',
 'script framework',
 'sensor reading',
 'serial port',
 'server',
 'server information',
 'service control',
 'service/process',
 'servo drive',
 'shellcode',
 'signature value',
 'software/program',
 'software/tool',
 'software/tool (malware)',
 'sql command',
 'sql stored procedure',
 'storage device',
 'string',
 'url',
 'user account',
 'variable state',
 'vulnerability',
 'vulnerability identifier',
 'web script',
 'windows registry key'])

In [8]:
gross_classifications = set([
"System",
"Module",
"Hardware",
"Device",
"PLC",
"POU",
"Identifier",
"Information",
"Credential",
"Data",
"Command",
"API",
"Code",
"Process",
"Configuration",
"Log",
"File",
"Payload",
"Software",
"Service",
"Firmware",
"Server",
"Communication",
"Protocol",
"Protocol Field",
"Connection Port",
"Vulnerability",
"Attack Technique"])

In [9]:
prompt = f"""
You are a cybersecurity analyst expert. Your job is to group a list of observable “classification” labels under gross classifications from MITRE ATT&CK® Enterprise, or if not matched to any existing gross classification - create a new category that matches the observable group.

---
### Input
   - A set of unique observable classifications:
     {sorted(unique_classifications)}

   - A set of gross classifications from MITRE ATT&CK® Enterprise:
     {sorted(gross_classifications)}

---
### Task
   - For each unique observable classifications, choose exactly one gross classification it best fits.
   - If it does not clearly belong to any of the gross classification labels given, assign it to a new category of your own choosing, appending “– new” to that category name (e.g. “Data Management – new”).

---
### Output
   - Return **only** a single JSON object.
   - Each key must be either one of the 28 gross classification label names or one of your “– new” categories.
   - Each value must be an array of the unique observable classifications assigned to that key.

---
### Response format (return *only* this JSON)

```json
{{
  "Code": [
    "script file"
  ],
  "Command": [
    "command string"
  ],
  "Identifier": [
    "ip address",
    "domain"
  ],
  "Data": [
    "file format",
    "file signature"
  ]
  ...
}}

"""

In [12]:
# !pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0


In [10]:
# 4. Call ChatGPT with model o3-mini-high
response = openai.ChatCompletion.create(
    model="o3-mini-2025-01-31",
    messages=[
        {"role": "system", "content": "You are a helpful cybersecurity expert."},
        {"role": "user", "content": prompt}
    ],
)

# 5. Print the JSON response
print(response.choices[0].message.content)

AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.